# Ensembling and Model Examination

### to ensemble with another classifier
need to do it on the training data, would need to hold out a very small portion that the cnns
dont get trained on at all, and use that for the random forest, 

features being pred1, pred2, incidence angle (so all of the ones without the incidence angle should be in the non inc angle model to preserve data)

Can probably do a little data augmentation on that as well (i.e. add a little noise)

### Individual Model LB performance (with clipping)
basic_model_reg = .1954 LB

inc_model_nonorm = .3188 LB

transfer_model = would love to know




In [17]:
# check correlations
import pandas as pd
import numpy as np

basic_mod = pd.read_csv("sub_full_basic_reg01_Fin.csv")
inc_mod = pd.read_csv("Inc_angle/sub_full_inc2_reg01_Fin.csv")
transfer_mod = pd.read_csv("Transfer/sub_full_transfer_incV3.csv")
test_data = pd.read_json("../Test_data/test.json")
ids = test_data["id"]
inc_angles = test_data["inc_angle"]

print("len inc_mod", len(inc_mod))
print("len inc angles", len(inc_angles))

print(basic_mod.head(5))
print(inc_mod.head(5))
print(transfer_mod.head(5))



len inc_mod 8424
len inc angles 8424
         id  is_iceberg
0  5941774d    0.020403
1  4023181e    0.882341
2  b20200e4    0.032158
3  e7f018bb    0.965896
4  4371c8c3    0.153121
         id  is_iceberg
0  5941774d    0.331743
1  4023181e    0.509914
2  b20200e4    0.532655
3  e7f018bb    0.819580
4  4371c8c3    0.492395
         id  is_iceberg
0  5941774d    0.000210
1  4023181e    0.277752
2  b20200e4    0.016809
3  e7f018bb    0.999883
4  4371c8c3    0.234150
0       34.966400
1       32.615072
2       37.505433
3       34.473900
4       43.918874
5       35.933020
6       42.414200
7       33.638300
8       41.107600
9       34.966400
10      35.782900
11      35.566871
12      33.185400
13      40.111298
14      39.698577
15      41.797300
16      43.603612
17      38.366264
18      44.988231
19      34.471800
20      44.678233
21      35.658800
22      35.408018
23      42.519800
24      45.247800
25      42.596474
26      45.366800
27      45.745633
28      44.066330
29      3

In [ ]:
#basic_mod["is_iceberg"] = np.clip(basic_mod["is_iceberg"], 0.01, 0.99)
#inc_mod["is_iceberg"] = np.clip(inc_mod["is_iceberg"], 0.01, 0.99)
#transfer_mod["is_iceberg"] = np.clip(transfer_mod["is_iceberg"], 0.01, 0.99)

#basic_mod.to_csv("basic_model_fin.csv", index=False)
#inc_mod.to_csv("inc_model_NOnorm_fin.csv", index=False)
#transfer_mod.to_csv("transfer_mod_fin.csv", index=False)

In [21]:
# mergeing the three together
merged = pd.merge(basic_mod, inc_mod, how='inner', on='id')
merged = pd.merge(merged, transfer_mod, how='inner', on='id')

merged.rename(index=str, columns={"is_iceberg_x": "basic",
                                  "is_iceberg_y": "inc",
                                 "is_iceberg": "transfer"}, inplace=True)

# attach inc angles
angle_ids = pd.concat([ids, inc_angles], axis=1)
merged = pd.merge(merged, angle_ids, how='inner', on='id')


merged_drop = merged.drop(['id'], axis=1)

# attach inc angles
#merged = pd.concat([merged, inc_angles], axis=1)

print(merged_drop.head(5))

corr = merged_drop.corr()
print(corr)

      basic       inc  transfer  inc_angle
0  0.020403  0.331743  0.000210  34.966400
1  0.882341  0.509914  0.277752  32.615072
2  0.032158  0.532655  0.016809  37.505433
3  0.965896  0.819580  0.999883  34.473900
4  0.153121  0.492395  0.234150  43.918874
              basic       inc  transfer  inc_angle
basic      1.000000  0.579747  0.684567  -0.039269
inc        0.579747  1.000000  0.597821  -0.019494
transfer   0.684567  0.597821  1.000000  -0.073140
inc_angle -0.039269 -0.019494 -0.073140   1.000000
